In [3]:
import pandas as pd
from datetime import timedelta
import datetime as dt
import time
from ipywidgets import HTML
from ipyleaflet import Map, Marker, Polyline, Icon, Popup, basemaps
from IPython.display import display, clear_output

# Load GTFS data
data_path = r"C:\Users\TULPAR\JupyterLab Projects\GTFS Project\dataset"

stops = pd.read_csv(f"{data_path}/stops.txt")
routes = pd.read_csv(f"{data_path}/routes.txt")
trips = pd.read_csv(f"{data_path}/trips.txt")
stop_times = pd.read_csv(f"{data_path}/stop_times.txt")
calendar = pd.read_csv(f"{data_path}/calendar.txt")
shapes = pd.read_csv(f"{data_path}/shapes.txt")
fakebuses=pd.read_csv(f"{data_path}/fakeBus.txt")

# Function to convert times
def convert_time(time_str):
    try:
        parts = time_str.split(':')
        hours, minutes, seconds = int(parts[0]), int(parts[1]), int(parts[2])
        if hours >= 24:
            hours -= 24
        return f"{hours:02}:{minutes:02}:{seconds:02}"
    except ValueError as e:
        print(f"Error converting time: {time_str} - {e}")
        return None

# Apply the conversion to arrival_time and departure_time
stop_times['arrival_time'] = stop_times['arrival_time'].apply(convert_time)
stop_times['departure_time'] = stop_times['departure_time'].apply(convert_time)
stop_times = stop_times.dropna(subset=['arrival_time', 'departure_time'])

# Simulation parameters
now_time = datetime.now()
current_time = datetime.strptime(now_time.strftime('%H:%M:%S'), '%H:%M:%S')
start_time = current_time
end_time = datetime.strptime('23:00:00', '%H:%M:%S')
time_step = timedelta(seconds=30)


def find_matching_service_ids(calendar):
    # Get today's date and weekday name
    today = dt.date.today()
    weekday_name = today.strftime("%A").lower()  

    today_str = today.strftime("%Y%m%d") 

    calendar['start_date'] = calendar['start_date'].astype(str)
    calendar['end_date'] = calendar['end_date'].astype(str)
    # Find rows where the current weekday has a '1' and the date is within the range
    matching_rows = calendar[
        (calendar[weekday_name] == 1) &
        (calendar['start_date'] <= today_str) &
        (calendar['end_date'] >= today_str)
    ]
    # Extract the matching service_ids
    matching_service_ids = matching_rows['service_id'].tolist()

    return matching_service_ids

today_id = find_matching_service_ids(calendar)
print("Matching service_ids for today:", today_id)
# Initialize the map centered on the city

center=(stops['stop_lat'].mean(),stops['stop_lon'].mean())
map = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=center, zoom_start=12)


# Add stop markers to the map initially
def add_stop_markers(stops, map):
    for stop in stops.itertuples():
        # Create a marker
        marker = Marker(location=[stop.stop_lat, stop.stop_lon], draggable=False)

        # Create a popup with the stop information
        popup_content = HTML()
        popup_content.value = f"Stop: {stop.stop_name}, Stop Number: {stop.stop_id}"
        popup = Popup(
            location=[stop.stop_lat, stop.stop_lon],
            child=popup_content,
            close_button=False,
            auto_close=False,
            close_on_escape_key=False
        )

        # Add the popup to the marker
        marker.popup = popup

        # Add the marker to the map
        map.add_layer(marker)
    
    return map
#map = add_stop_markers(stops, map)


def create_lines(shapes, map):
    shape_locations = []
    for shape in shapes.itertuples():
        shape_locations.append([shape.shape_pt_lat, shape.shape_pt_lon])
    
    if len(shape_locations) > 1:
        polyline = Polyline(
            locations=shape_locations,
            color="red",
            fill=False,
            weight=2,
            opacity=1,
            dash_array='5 ,5'
        )
        map.add_layer(polyline)
    
    return map
map=create_lines(shapes,map)

# Dictionary to store bus markers
bus_markers = {}
merged_df = pd.merge(trips, stop_times, on='trip_id', how='inner')
#print(merged_df)
# Initialize buses
buses = [{
    'trip_id': trip.trip_id,
    'route_id': trip.route_id,
    'current_stop_sequence': trip.stop_sequence,
    'stop_time': trip.arrival_time
    
} for trip in merged_df.itertuples()]
display(buses)

buses = [{
    'trip_id': trip.trip_id,
    'route_id': trip.route_id,
    'current_stop_sequence': trip.stop_sequence,
    'stop_time': trip.arrival_time
    
} for trip in fakebuses.itertuples()]
display(buses)
def create_bus_markers(buses, stops, stop_times, map):
    # Add new markers 
    for bus in buses:
        bus_trip_id = bus['trip_id']
        current_stop_sequence = bus['current_stop_sequence']
        if current_stop_sequence > 0:
            stop_times_for_trip = stop_times[stop_times['trip_id'] == bus_trip_id]
            if current_stop_sequence < len(stop_times_for_trip):
                current_stop_id = stop_times_for_trip.iloc[current_stop_sequence]['stop_id']
                stop = stops[stops['stop_id'] == current_stop_id]
                if not stop.empty:
                    #icon_url ='https://www.flaticon.com/free-icon/bus-stop_3448339?term=bus+map&page=1&position=2&origin=search&related_id=3448339'
                    marker = Marker(
                        location=[stop['stop_lat'].values[0], stop['stop_lon'].values[0]],
                        #icon=Icon(icon_url=icon_url, icon_size=[30, 30]),
                        title=f"Bus on route {bus['route_id']} at stop {stop['stop_name'].values[0]}"
                    )
                    popup_content = HTML()
                    popup_content.value =(f"Bus on route {bus['route_id']} at stop {stop['stop_name'].values[0]}")
                    popup = Popup(
                    location=[stop['stop_lat'].values[0], stop['stop_lon'].values[0]],
                    child=popup_content,
                    close_button=False,
                    auto_close=False,
                    close_on_escape_key=False
                    )
                    
                    marker.popup=popup
                    map.add_layer(marker)
                    bus_markers[bus_trip_id] = marker

                    
    # Remove old markers
    for marker in bus_markers.values():
        map.remove_layer(marker)
    bus_markers.clear()
    return map
    
map=create_bus_markers(buses, stops, stop_times, map)

map.save('initial_map.html')
# Load the initial map with ipyleaflet
map

#Updating the bus positions
def update_bus_positions(buses, stop_times, current_time):
    updated_buses = []
    for bus in buses:
        trip_id = bus['trip_id']
        stop_times_for_trip = stop_times[stop_times['trip_id'] == trip_id]
        if not stop_times_for_trip.empty:
            current_stop_sequence = bus['current_stop_sequence']
            if current_stop_sequence < len(stop_times_for_trip):
                current_stop = stop_times_for_trip.iloc[current_stop_sequence]
                arrival_time = datetime.strptime(current_stop['arrival_time'], '%H:%M:%S')
                if current_time >= arrival_time:
                    bus['current_stop_sequence'] += 1
            if bus['current_stop_sequence'] < len(stop_times_for_trip):
                updated_buses.append(bus)
    return updated_buses


# Simulation loop
while current_time <= end_time:
    buses = update_bus_positions(buses, stop_times, current_time)
    map = create_bus_markers(buses, stops, stop_times, map)
    current_time += time_step
    print(current_time)
    clear_output(wait=True)
    display(map)
    time.sleep(30)
    print("Map updated")
    

NameError: name 'datetime' is not defined